In [1]:
import pandas as pd

df = pd.read_json('result_data.json')

KeyboardInterrupt: 

In [ ]:
from datetime import datetime

def change_time(dt):
    return datetime.fromtimestamp(dt//1000)

df['fetchdate_orig'] = df['fetchdate_orig'].apply(change_time)
df['fetchdate_check'] = df['fetchdate_check'].apply(change_time)

In [ ]:
start_date = datetime.strptime("24-02-2022", "%d-%m-%Y")
end_date = datetime.strptime("24-03-2022", "%d-%m-%Y")
df_war = df[(df['fetchdate_check'] >= start_date) & (df['fetchdate_check'] <= end_date)]

In [ ]:
df_test = df_war['page_title'].to_frame().reset_index()
df_test.drop(['index'], axis=1)

In [ ]:
import re
import pandas as pd
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
import numpy as np
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords

In [ ]:
stopwords = stopwords.words('russian')

In [ ]:
bad_words = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_{|}~\"\-]+"
morph = MorphAnalyzer()

def token_only(text):
    text = re.sub(bad_words,' ',text)
    tokens = [word.lower() for sent in sent_tokenize(text) for word in word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        token = token.strip()
        token = morph.normal_forms(token)[0]
        filtered_tokens.append(token)
    return filtered_tokens


dfclall = df_test['page_title']
stopwords.extend(["и", "в", "во", "не", "что", "он", "на", "я", "с", "со", "как", "а", "то", "все", "она", "так", "его",
"но", "да", "ты", "к", "у", "же", "вы", "за", "бы", "по", "только", "ее", "мне", "было", "вот", "от", "меня", "еще",
"нет", "о", "из", "ему", "теперь", "когда", "даже", "ну", "вдруг", "ли", "если", "уже", "или", "ни", "быть", "был",
"него", "до", "вас", "нибудь", "опять", "уж", "вам", "ведь", "там", "потом", "себя", "ничего", "ей", "может", "они",
"тут", "где", "есть", "надо", "ней", "для", "мы", "тебя", "их", "чем", "была", "сам", "чтоб", "без", "будто", "чего",
"раз", "тоже", "себе", "под", "будет", "ж", "тогда", "кто", "этот", "того", "потому", "этого", "какой", "совсем",
"ним", "здесь", "этом", "один", "почти", "мой", "тем", "чтобы", "нее", "сейчас", "были", "куда", "зачем", "всех",
"никогда", "можно", "при", "наконец", "два", "об", "другой", "хоть", "после", "над", "больше", "тот", "через", "эти",
"нас", "про", "всего", "них", "какая", "много", "разве", "три", "эту", "моя", "впрочем", "хорошо", "свою", "этой",
"перед", "иногда", "лучше", "чуть", "том", "нельзя", "такой", "им", "более", "всегда", "конечно", "всю", "между", 'иза'])


tfidf_vectorizer = TfidfVectorizer(smooth_idf=True,max_df=0.6,min_df=0.01,max_features=100000,
                                   stop_words=stopwords, use_idf=True,tokenizer=token_only, ngram_range=(1,3))
tfidf_matrix = tfidf_vectorizer.fit_transform(dfclall)

In [ ]:
inertia = []
for k in range(100,3000,100):
    mbk  = MiniBatchKMeans(n_clusters=k,init='random', random_state=1).fit(tfidf_matrix)
    inertia.append(np.sqrt(mbk.inertia_))
plt.plot(range(100,3000,100),inertia,marker='s')
plt.xlabel('K')
plt.ylabel('(C_k)')
plt.show()

In [ ]:
mbk = MiniBatchKMeans(n_clusters=600,init='random').fit(tfidf_matrix)
y_kmeansMBK = mbk.predict(tfidf_matrix)
Num = []
Num = [pt for pt in y_kmeansMBK]
df2 = {"Num_Cluster": Num}
dfMBK = pd.DataFrame(df2)

df = pd.concat([df_test,dfMBK], axis=1)

In [ ]:
df

In [ ]:
df.sort_values(by="Num_Cluster").to_csv('titles_clusters.csv')